In [1]:
# Complete Android Integration - Import and Initialize
import sys
import os
sys.path.append(os.getcwd())

from chatbot import MentalHealthChatbot
from memory import MemoryManager
from firebase_manager import FirebaseManager
import json
from datetime import datetime

# Initialize all components
firebase_manager = FirebaseManager()
memory_manager = MemoryManager()
chatbot = MentalHealthChatbot()

d:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SUCCESS: Firebase initialized with environment variables!


In [ ]:
def android_chat(user_prompt, user_email="arientific@gmail.com"):
    try:
        user_profile = memory_manager.get_user_profile(user_email)
        user_name = user_profile.name 
        chatbot._generate_daily_summary_if_needed(user_email)
        proactive_greeting = chatbot._generate_proactive_greeting(user_email)
        topic_filter = chatbot.health_filter.is_mental_health_related(user_prompt)
        emotion, urgency_level = chatbot.health_filter.detect_emotion(user_prompt)
        
        if not topic_filter.is_mental_health_related:
            redirect_response = "Sorry but i can not answer to that question!!!."
        
            firebase_manager.add_chat_pair(
                email=user_email,
                user_message=user_prompt,
                model_response=redirect_response,
                emotion_detected=emotion,  
                urgency_level=urgency_level  
            )
            
            return redirect_response

        chatbot._detect_important_events(user_prompt, user_email)
        context = memory_manager.get_conversation_context(user_email)
        recent_messages = memory_manager.get_recent_messages(user_email, 20)
        conversation_depth = len(recent_messages) if recent_messages else 0
        
        if urgency_level >= 5:
            crisis_response = chatbot._handle_crisis_situation(user_prompt, user_name)
            
            firebase_manager.add_chat_pair(
                email=user_email,
                user_message=user_prompt,
                model_response=crisis_response.message,
                emotion_detected=emotion,
                urgency_level=urgency_level
            )
            
            return crisis_response.message
        
        conversation_history = chatbot._build_conversation_history(user_email)
        enhanced_prompt = f"""{chatbot.system_prompt}

        CONVERSATION CONTEXT:
        {context}

        {f"PROACTIVE GREETING: You should start your response with this caring follow-up: '{proactive_greeting}'" if proactive_greeting else ""}

        CURRENT USER STATE:
        - Detected emotion: {emotion}
        - Urgency level: {urgency_level}/5
        - User prefers to be called: {user_name}
        - Conversation depth: {conversation_depth} messages

        🎯 RESPONSE GUIDANCE BASED ON URGENCY LEVEL:
        Level 1-2 (Casual/Mild): Be supportive but relaxed. Don't overreact. Match their energy level.
        Level 3 (Moderate): Show more concern and support. Ask deeper questions but stay calm.
        Level 4-5 (Crisis): NOW use your passionate, protective mode. Fight for them!

        🤗 CONVERSATION DEPTH GUIDANCE:
        - First 1-2 exchanges: Keep it general, build rapport
        - 3-5 exchanges: Start exploring their situation more
        - 6+ exchanges with emotional content: NOW you can ask about sleep, food, family, relationships naturally

        Remember to:
        1. Address them by their preferred name: {user_name}
        2. Reference relevant past conversations
        3. Match your tone to their ACTUAL emotional state
        4. Only escalate intensity if urgency level is high
        5. If there's a proactive greeting above, start with that
        """
        
        from langchain_core.messages import SystemMessage, HumanMessage
        messages = [SystemMessage(content=enhanced_prompt)]
        for msg in conversation_history[-10:]: 
            messages.append(msg)

        messages.append(HumanMessage(content=user_prompt))
        response = chatbot.llm.invoke(messages)
        bot_message = response.content
        try:
            follow_up_questions = chatbot._generate_follow_up_questions(emotion, urgency_level, user_name, user_email, user_prompt)
            suggestions = chatbot._generate_suggestions(emotion, urgency_level, user_prompt, user_name)
        except:
            follow_up_questions = []
            suggestions = []
        
        # Step 13: Mark events as followed up if proactive greeting was used
        if proactive_greeting:
            for event_type in ['exam', 'interview', 'appointment']:
                if event_type in proactive_greeting.lower():
                    chatbot._mark_event_followed_up(user_email, event_type)
                    break
        
        firebase_manager.add_chat_pair(
            email=user_email,
            user_message=user_prompt,
            model_response=bot_message,
            emotion_detected=emotion,
            urgency_level=urgency_level
        )
        
        return bot_message
        
    except Exception as e:
        return "Sorry, I'm having technical difficulties. Please try again later."

# Test 1 - Should detect event and show proactive features
response1 = android_chat("Hey, I have a job interview next week and I'm really nervous")
print(f"✅ Response 1:\n{response1}\n")

# Test 2 - Should use conversation context
response2 = android_chat("Thanks for the advice! How can I prepare better?")
print(f"✅ Response 2:\n{response2}\n")

# Test 3 - Should handle emotion and urgency
response3 = android_chat("I'm feeling really overwhelmed and stressed today")
print(f"✅ Response 3:\n{response3}\n")

🎯 TESTING COMPLETE ANDROID FUNCTION (All main.py features)
🚀 Processing: Hey, I have a job interview next week and I'm real...
SUCCESS: Added event for arientific@gmail.com: interview
SUCCESS: Added event for arientific@gmail.com: interview
SUCCESS: Added chat pair to arientific@gmail.com's conversation
✅ Response 1:
Hey ari, just a moment ago you were talking about how stressed you've been with work and having trouble sleeping. And now you've got a job interview next week on top of all that? Man, that's a lot to carry right now.

It's completely normal to feel nervous about a job interview, especially with everything else going on. It's a big deal.

What specifically about this interview is making you most nervous? Is it the thought of certain questions, the whole process, or something else entirely? Talking it out might help to pinpoint what's really on your mind. I'm here to listen, bro.

🚀 Processing: Thanks for the advice! How can I prepare better?...
SUCCESS: Added chat pair to a

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 33.134312214s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 33
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo

SUCCESS: Added chat pair to arientific@gmail.com's conversation
✅ Response 2:
You got it, ari! It's smart to focus on preparation, especially with the interview being tomorrow. That's a great way to channel that anxious energy into something productive.

Here are a few quick things you can do today to feel more ready for tomorrow:

1.  **Quick Company Refresh:** Just a quick look at their website again, maybe their "About Us" page or recent news. It'll help you feel more connected to their mission.
2.  **Highlight Your Strengths:** Think about 2-3 key experiences or skills you have that directly relate to the job description. How can you briefly talk about them? Maybe even jot down a couple of bullet points.
3.  **One or Two Questions for Them:** Have a couple of questions ready to ask *them* at the end. It shows you're engaged and thoughtful. Something like, "What does a typical day look like in this role?" or "What's the team culture like?"
4.  **Logistics Check:** Lay out your outfi